In [2]:
# import standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Input, Dense
from keras.models import Model
from fastai.vision.learner import cnn_learner
from fastai.vision import *
import os
import glob

Using TensorFlow backend.

Bad key "text.kerning_factor" on line 4 in
/Users/mitch/opt/anaconda3/envs/ml/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [12]:
# load train data
path = '../data/'
working_dir = os.getcwd() + '/'
train_data = pd.read_csv(path+'MoreManipulatedData.csv')

print(working_dir)

/Users/mitch/Documents/4B_university/machine_intelligence/SYDE522_project/model_training/


In [15]:
print(train_data.BUCKET.unique())
print(train_data.shape)

[' 1-200   ' ' 601+    ' ' 201-600 ' '0']
(33498, 18)


In [ ]:
# Splitting into train test and validate


In [16]:
train_data.head(5)

,PLAYER_ID,PLAYER_URL,POS,TOTAL_JUNIOR_GP,TOTAL_JUNIOR_GOALS,TOTAL_JUNIOR_ASSISTS,TOTAL_JUNIOR_POINTS,TOTAL_JUNIOR_PIM,TOTAL_JUNIOR_PLUS_MINUS,LEAGUE,FIRST_JUNIOR_YEAR,TOTAL_JUNIOR_YEARS,BUCKET,DOB,NATIONALITY,HEIGHT_CM,WEIGHT_KG,SHOOTS
0,103143,https://www.eliteprospects.com/player/103143/...,LW,87,50,57,107,209,0,OHL,1981-1982,2,1-200,1963-02-10,Canada,180,84,L\
1,106210,https://www.eliteprospects.com/player/106210/...,D,73,15,39,54,123,0,NCAA,1981-1982,2,1-200,1963-01-27,USA,193,95,-\
2,106207,https://www.eliteprospects.com/player/106207/...,F,21,7,11,18,22,0,NCAA,1981-1982,1,1-200,1959-08-24,USA/Canada,183,86,-\
3,106205,https://www.eliteprospects.com/player/106205/...,D,70,8,37,45,116,0,NCAA,1981-1982,2,1-200,1961-05-03,USA,188,86,L\
4,101953,https://www.eliteprospects.com/player/101953/...,F,118,47,65,112,174,0,NCAA,1981-1982,3,1-200,1960-10-01,Canada,191,96,-\


In [6]:
data = ImageList.from_df(path=working_dir, df=train_data, cols='train_file')
data = data.split_by_rand_pct(valid_pct=0.2, seed=1)
data = data.label_from_df(cols='Category')
data = data.databunch()


In [7]:
learner = cnn_learner(data, models.resnet50, metrics=[error_rate, accuracy])

In [8]:
learner.fit_one_cycle(10)
learner.save('Trial_3_5epochs_resnet50')

epoch,train_loss,valid_loss,error_rate,accuracy,time
0,0.965198,0.299376,0.104444,0.895556,10:45
1,0.518760,0.277674,0.086667,0.913333,10:55
2,0.350166,0.209985,0.066667,0.933333,11:09
3,0.231804,0.233539,0.070000,0.930000,11:15
4,0.158007,0.195520,0.065556,0.934444,11:41
5,0.113653,0.199095,0.058889,0.941111,10:30
6,0.083182,0.193795,0.053333,0.946667,12:42


RuntimeError: DataLoader worker (pid 10707) is killed by signal: Unknown signal: 0. 

In [9]:
def cnn_prodictor(img):
    pred = learner.predict(img)
    return pred[0]

In [10]:
test_path = working_dir + 'test/test/'
test_images = ImageList.from_folder(path=test_path)

In [11]:
# def random_predictor(img):
#     # this must be your glorious classifier and not a random predictor
#     return train_data.sample(n=1).Category.iloc[0]


test_files = glob.glob(test_path+'*.png')
test_file_id = [os.path.basename(test_file) for test_file in test_files]

# test_submission = pd.DataFrame({'Id': test_file_id, 'Category': [cnn_prodictor(imread(test_path+test_img_id)) for test_img_id in test_file_id]})

# test_submission = pd.DataFrame({'Id': test_file_id, 'Category': [cnn_prodictor(imread(test_path+test_img_id)) for test_img_id in test_file_id]})
test_submission = pd.DataFrame({'Id': test_file_id, 'Category':[cnn_prodictor(img) for img in test_images]})

test_submission.to_csv('submission.csv', index=False)